In [1]:
# !wget https://huggingface.co/datasets/mesolitica/ms-wikipedia-choice/resolve/main/qa-ms-wikipedia.jsonl

In [2]:
import json
import random

In [3]:
data = []
with open('qa-ms-wikipedia.jsonl') as fopen:
    for l in fopen:
        data.append(json.loads(l))

In [4]:
len(data)

8217

In [5]:
data[0]['qa']['qa'][0]

{'question': 'Apakah nama-nama sebahagian besar wilayah di Zimbabwe?',
 'A': 'Mashonaland dan Matabeleland',
 'B': 'Jagung, kapas, tembakau, gandum, kopi dan tebu',
 'C': 'Syarikat Afrika Selatan British Pioneer Column',
 'D': 'Perang Matabele Pertama',
 'answer': 'A'}

In [6]:
indices = {i for i in range(len(data))}

In [7]:
from tqdm import tqdm

pairs = []
for i in tqdm(range(len(data))):
    query = data[i]['paragraph']
    pos = []
    for s in data[i]['qa']['qa']:
        pos.append(s['question'])
    
    neg_i = random.choice(list(indices - {i}))
    neg = []
    for s in data[neg_i]['qa']['qa']:
        neg.append(s['question'])
        
    pairs.append({
        'query': query,
        'pos': pos,
        'neg': neg
    })
    
    for s in data[i]['qa']['qa']:
        try:
            pos = [query]
            neg_i = random.choice(list(indices - {i}))
            neg = [data[neg_i]['paragraph']]

            pairs.append({
                'query': s['question'],
                'pos': pos,
                'neg': neg
            })
        except:
            pass

100%|█████████████████████████████████████| 8217/8217 [00:03<00:00, 2511.51it/s]


In [8]:
pairs[1]

{'query': 'Apakah nama-nama sebahagian besar wilayah di Zimbabwe?',
 'pos': ['pertanian yang utama adalah jagung, kapas, tembakau, gandum, kopi dan tebu. Zimbabwe adalah kerajaan berpusat dan terbahagi kepada lapan wilayah dan dua bandar raya dengan status wilayah untuk tujuan pentadbiran. Setiap wilayah pula mempunyai ibu negerinya yang tersendiri di mana perniagaan rasmi dijalankan. Nama-nama sebahagian besar wilayah dihasilkan daripada perpecahan Mashonaland dan Matabeleland semasa penjajahan: Mashonaland adalah wilayah yang pertama sekali diduduki Syarikat Afrika Selatan British Pioneer Column dan Matabeleland pula ialah wilayah yang ditakluki semasa Perang Matabele Pertama. Ini sejajar dengan wilayah pra-kolonial orang Shona dan orang Matabele, walaupun terdapat etnik minoriti yang signifikan di kebanyakan wilayah. Setiap provinsi diketuai oleh seorang Gabenor Wilayah, yang dilantik oleh Presiden. Pemerintah wilayah dikendalikan oleh Pentadbir Provinsi, yang dilantik oleh Suruhanj

In [9]:
len(pairs)

50010

In [10]:
with open('qa-ms-wikipedia-pair.jsonl', 'w') as fopen:
    for l in pairs:
        if not len(l['pos']) or not len(l['neg']):
            continue
        fopen.write(f'{json.dumps(l)}\n')